# Parsing PDFs Homework

With the power of pdfminer, pytesseract, Camelot, and Tika, let's analyze some documents!

> If at any point you think, **"I'm close enough, I'd just edit the rest of it in Excel"**: that's fine! Just make a note of it.

## A trick to use again and again

### Approach 1

Before we get started: when you want to take the first row of your data and set it as the header, use this trick.

In [1]:
#import libraries
import pandas as pd

In [2]:
df = pd.DataFrame([
    [ 'fruit name', 'likes' ],
    [ 'apple', 15 ],
    [ 'carrot', 3 ],
    [ 'sweet potato', 45 ],
    [ 'peach', 12 ],
])

In [3]:
df

,0,1
0,fruit name,likes
1,apple,15
2,carrot,3
3,sweet potato,45
4,peach,12


In [4]:
# Set the first column as the columns
df.columns = df.loc[0]

In [5]:
# Drop the first row
df = df.drop(0)

In [6]:
df

,fruit name,likes
1,apple,15
2,carrot,3
3,sweet potato,45
4,peach,12


🚀 Done!

### Approach 2

Another alternative is to use `.rename` on your columns and just filter out the columns you aren't interested in. This can be useful if the column name shows up multiple times in your data for some reason or another.

In [7]:
# Starting with the same-ish data...
df = pd.DataFrame([
    [ 'fruit name', 'likes' ],
    [ 'apple', 15 ],
    [ 'carrot', 3 ],
    [ 'fruit name', 'likes' ],
    [ 'sweet potato', 45 ],
    [ 'peach', 12 ],
])

In [8]:
df

,0,1
0,fruit name,likes
1,apple,15
2,carrot,3
3,fruit name,likes
4,sweet potato,45
5,peach,12


In [9]:
df = df.rename(columns={
    0: 'fruit name',
    1: 'likes'
})

In [10]:
df = df[df['fruit name'] != 'fruit name']

In [11]:
df

,fruit name,likes
1,apple,15
2,carrot,3
4,sweet potato,45
5,peach,12


🚀 Done!

### Useful tips about coordinates

If you want to grab only a section of the page [Kull](https://jsoma.github.io/kull/#/) might be helpful in finding the coordinates.

> **Alternatively** run `%matplotlib notebook` in a cell. Afterwards, every time you use something like `camelot.plot(tables[0]).show()` it will get you nice zoomable, hoverable versions that include `x` and `y` coordinates as you move your mouse.

Coordinates are given as `"left_x,top_y,right_x,bottom_y"` with `(0,0)` being in the bottom left-hand corner.

Note that all coordinates are strings, for some reason. It won't be `[1, 2, 3, 4]` it will be `['1,2,3,4']`

# The homework

This is **mostly Camelot work**, because I don't really have any good image-based PDFs to stretch your wings on tesseract. If you know of any, let me know and I can put together another couple exercises.

## Prison Inmates

Working from [InmateList.pdf](InmateList.pdf), save a CSV file that includes every inmate.

* Make sure your rows are *all data*, and you don't have any people named "Inmate Name."


In [12]:
import camelot

In [13]:
inmate = camelot.read_pdf('sources/InmateList.pdf', flavor = 'stream', pages = '1-end', table_areas = ["17,732,358,105"])

In [14]:
dfs = [table.df for table in inmate]

In [15]:
inmate_df = pd.concat(dfs, ignore_index = True)

In [16]:
inmate_df = inmate_df.rename(columns={
    0: 'ICN #',
    1: 'Inmate Name',
    2: 'Facility',
    3: 'Booking Date'
})

In [17]:
inmate_df = inmate_df[(inmate_df['ICN #'] != 'ICN #')]

In [18]:
inmate_df.to_csv("outputs/inmate_list.csv",index = False)

## WHO resolutions

Using [A74_R13-en.pdf](A74_R13-en.pdf), what ten member countries are given the highest assessments?

* You might need to have two separate queries, and combine the results: that last page is pretty awful!
* Always rename your columns
* Double-check that your sorting looks right......
* You can still get the answer even without perfectly clean data

In [19]:
who = camelot.read_pdf('sources/A74_R13-en.pdf', flavor = 'stream', pages = '1-5')
who_6 = camelot.read_pdf('sources/A74_R13-en.pdf', flavor = 'stream', pages = '6', table_areas = ["75,744,388,644"])

In [20]:
who_dfs = [table.df for table in who]

In [21]:
who_df = pd.concat(who_dfs, ignore_index = True)

In [22]:
who_df

,0,1
0,Members and,WHO scale
1,Associate Members,for 2022–2023
2,,%
3,Afghanistan,0.0070
4,Albania,0.0080
...,...,...
210,Uzbekistan,0.0320
211,Vanuatu,0.0010
212,Venezuela (Bolivarian Republic of),0.7280
213,Viet Nam,0.0770


In [23]:
who_df = who_df.append(who_6[0].df)

In [24]:
who_df.head()

,0,1
0,Members and,WHO scale
1,Associate Members,for 2022–2023
2,,%
3,Afghanistan,0.0070
4,Albania,0.0080


In [25]:
who_df = who_df[(who_df[0] != 'Members and') & (who_df[0] != 'Associate Members') & (who_df[1] != 'WHO scale') & (who_df[1] != '%') & (who_df[1] != 'for 2022-2023')& (who_df[0] != 'TOTAL')]

In [26]:
who_df = who_df.rename(columns = {
    0 : 'member_state',
    1 : 'assessment'
}).reset_index(drop = True)

In [27]:
who_df.assessment = pd.to_numeric(who_df.assessment)

In [28]:
who_df.sort_values(by = 'assessment', ascending=False).head(10)

,member_state,assessment
193,United States of America,22.0000
35,China,12.0058
88,Japan,8.5645
68,Germany,6.0904
191,Northern Ireland,4.5673
64,France,4.4273
86,Italy,3.3072
23,Brazil,2.9482
31,Canada,2.7342
148,Russian Federation,2.4052


## The Avengers

Using [THE_AVENGERS.pdf](THE_AVENGERS.pdf), approximately how many lines does Captain America have as compared to Thor and Iron Man?

* Character names only: we're only counting `IRON MAN` as Iron Man, not `TONY`.
* Your new best friend might be `\n`
* Look up `.count` for strings

In [29]:
from pdfminer.high_level import extract_text

In [30]:
avengers = extract_text('sources/THE_AVENGERS.pdf')

In [31]:
avengers

'Marvel’s THE AVENGERS\n\nWritten By\n\nJoss Whedon\n\nStory By\nZak Penn and Joss Whedon\n\nBased on the characters appearing in the comic books\nPublished by MARVEL Comics\n\nTranscribed to PDF using:\nCeltx Studio | 2012\n\nComposed In Celtx Studio\nWith the help of\nSiOmniaFicta’s Script\non fanfiction.net\n\nENTERTAINMENT PURPOSES ONLY\n\n\x0c1\n\n2\n\n3\n\n1\n\nEXT. UNKNOWN AREA OF SPACE\n\nSpace, a floating staircase among the rocks...THE OTHER\nspeaks to a DISEMBODIED FIGURE in a chair.\n\nTHE OTHER hands LOKI a scepter with a glowing blue gem...\n\nTHE OTHER (V.O)\n\nThe Tesseract has awakened. It is\non a little world, a human world.\nThey would wield its power.\n\nTHE OTHER (V.O)\nBut our ally knows its workings as\nthey never will. He is ready to\nlead. And our force...\n\nTHE OTHER (V.O)\n\n...our Chitauri, will follow. The\nworld will be his...the universe,\nyours. And the humans, what can\nthey do but burn?\n\nThe CHITAURI put on their face mask and get ready.\n\nA 3d mo

In [32]:
cap_lines = avengers.count("\nCAPTAIN AMERICA\n")
cap_lines

35

In [33]:
thor_lines = avengers.count("\nTHOR\n")
thor_lines

47

In [34]:
ironman_lines = avengers.count("\nIRON MAN\n")
ironman_lines

19

In [35]:
f"Captain America has {cap_lines-ironman_lines} more lines than Iron Man, but {abs(cap_lines-thor_lines)} fewer lines than Thor."

'Captain America has 16 more lines than Iron Man, but 12 fewer lines than Thor.'

In [36]:
#to count words after: capture the lines after captain america 
#regex that extracts this part of the text

## COVID data

Using [covidweekly2721.pdf](covidweekly2721.pdf), what's the total number of tests performed in Minnesota? Use the Laboratory Test Rates by County of Residence chart.

* You COULD pull both tables separately OR you could pull them both at once and split them in pandas.
* Remember you can do things like `df[['name','age']]` to ask for multiple columns

In [37]:
covid = camelot.read_pdf('sources/covidweekly2721.pdf', flavor = 'lattice', pages = '6')

In [38]:
covid = covid[1].df

In [39]:
covid.columns = covid.loc[0]


In [40]:
covid = covid.drop(0)

In [41]:
covid_1 = covid.iloc[:,[0,1,2]]
covid_2 = covid.iloc[:,[3,4,5]]

In [42]:
covid = covid_1.append(covid_2).reset_index(drop = True)

In [43]:
covid.loc[covid['County']=='Benton'] = [['Benton', '77,865', '19,574']]

In [44]:
covid

,County,Number of Tests,Cumulative Rate
0,Aitkin,"19,204","12,128"
1,Anoka,"545,958","15,714"
2,Becker,"59,238","17,540"
3,Beltrami,"60,345","13,085"
4,Benton,"77,865","19,574"
...,...,...,...
83,Wilkin,"7,973","12,570"
84,Winona,"106,625","20,970"
85,Wright,"194,085","14,621"
86,Yellow Medicine,"19,972","20,239"


## Theme Parks

Using [2019-Theme-Index-web-1.pdf](2019-Theme-Index-web-1.pdf), save a CSV of the top 10 theme park groups worldwide.

* You can clean the results or you can restrict the area the table is pulled from, up to you

In [45]:
themepark = camelot.read_pdf('sources/2019-Theme-Index-web-1.pdf', pages = '11', flavor = 'stream', table_areas = ["12,469,400,291"])

In [46]:
themepark = themepark[0].df

In [47]:
themepark = themepark.rename(columns = {
    0: 'rank',
    1: 'group_name',
    2: '%_change',
    3: 'attendance_2019',
    4: 'attendance_2018'
})

In [48]:
themepark.to_csv('outputs/top10themeparkgroups.csv', index = False)

## Hunting licenses

Using [US_Fish_and_Wildlife_Service_2021.pdf](US_Fish_and_Wildlife_Service_2021.pdf) and [a CSV of state populations](http://goodcsv.com/geography/us-states-territories/), find the states with the highest per-capita hunting license holders.

In [49]:
hunt = camelot.read_pdf('sources/US_Fish_and_Wildlife_Service_2021.pdf', pages = '1', flavor = 'lattice', strip_text = " .\n,") 

In [50]:
hunt = hunt[0].df

In [51]:
hunt = hunt.rename(columns = {
    0 : 'state',
    1 : 'paid_hunt_lholders',
    2 : 'resident_hunt_ltps',
    3 : 'non-res_hunt_ltps',
    4 : 'total_hunt_ltps',
    5 : 'cost-resident_hunt_ltps',
    6 : 'cost-non-res_hunt_ltps',
    7 : 'gross_cost_hunt_licenses'
}).drop([0,57])

In [52]:
hunt

,state,paid_hunt_lholders,resident_hunt_ltps,non-res_hunt_ltps,total_hunt_ltps,cost-resident_hunt_ltps,cost-non-res_hunt_ltps,gross_cost_hunt_licenses
1,AK,93559,423501,59235,482736,$4859356,$9046715,$13906071
2,AL,452400,601683,45397,647080,$9700295,$6715734,$16416029
3,AR,343300,349098,150728,499826,$7851601,$11271653,$19123254
4,AS,0,0,0,0,$0,$0,$0
5,AZ,302383,464607,88708,553315,$13931397,$5968169,$19899566
6,CA,262009,949540,25535,975075,$20270941,$1114625,$21385566
7,CO,296609,481253,120954,602207,$13270492,$49732815,$63003307
8,CT,32052,101377,4543,105920,$1855046,$372108,$2227154
9,DC,0,0,0,0,$0,$0,$0
10,DE,15619,45997,6044,52041,$899857,$499867,$1399724


In [53]:
hunt['total_license_percap'] = pd.to_numeric(hunt['total_hunt_ltps']) / pd.to_numeric(hunt['paid_hunt_lholders'])

In [54]:
hunt.sort_values(by = 'total_license_percap', ascending = False).head(5)

,state,paid_hunt_lholders,resident_hunt_ltps,non-res_hunt_ltps,total_hunt_ltps,cost-resident_hunt_ltps,cost-non-res_hunt_ltps,gross_cost_hunt_licenses,total_license_percap
44,RI,7514,49019,4705,53724,$607820,$133965,$741785,7.149854
54,WI,666670,3965367,236639,4202006,$28526992,$7884672,$36411664,6.302977
16,ID,275244,1412039,248610,1660649,$11465795,$18704191,$30169986,6.033370
35,NJ,68767,281220,121963,403183,$6328826,$1152802,$7481628,5.863030
1,AK,93559,423501,59235,482736,$4859356,$9046715,$13906071,5.159696
